# GCTB SBayesR

https://cnsgenomics.com/software/gctb/#SBayesRTutorial



In [3]:
import numpy as np
import pandas as pd
import subprocess

def run_bash_command(command):
    """
    Execute a bash command and print the intermediate output in real-time.

    Parameters:
    - command: A string containing the bash command to be executed.
    """
    # Run the command and capture the output.
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Display standard output in real-time.
    for line in process.stdout:
        print(line, end='')

    # Display standard error in real-time.
    for line in process.stderr:
        print(line, end='')

    process.wait()  # Wait for the process to complete.

In [ ]:
# %%bash
# wget https://cnsgenomics.com/software/gctb/download/gctb_2.05beta_Linux.zip
# unzip gctb_2.05beta_Linux.zip -d gctb_2.05beta_Linux
# gctb_2.05beta_Linux/gctb_2.05beta_Linux/gctb

In [28]:
!pwd

/home/vrothenberg/teams/group-23/UCSD_CSE284_FinalProject


In this command sequence, the genotype data is being re-filtered by removing low minor allele frequency variants, specifically focusing on chromosome 1, using PLINK, resulting in a new filtered dataset stored in the directory "ld_pruned_filtered".

In [43]:
%%bash
# Re-filtering genotype data
rm -rf ld_pruned_filtered
mkdir ld_pruned_filtered
plink --bfile data/bmi/ld_pruned_genotype_data/LD_pruned_PLINK/P50_round2_LD_pruned_3473 \
--chr 1 \
--make-bed \
--maf 0.0001 \
--out ld_pruned_filtered/chr1

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ld_pruned_filtered/chr1.log.
Options in effect:
  --bfile data/bmi/ld_pruned_genotype_data/LD_pruned_PLINK/P50_round2_LD_pruned_3473
  --chr 1
  --maf 0.0001
  --make-bed
  --out ld_pruned_filtered/chr1

515699 MB RAM detected; reserving 257849 MB for main workspace.
13696 out of 128401 variants loaded from .bim file.
3473 people (0 males, 0 females, 3473 ambiguous) loaded from .fam.
Ambiguous sex IDs written to ld_pruned_filtered/chr1.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 3473 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%

We can see the different files being generated.

In [44]:
!ls ld_pruned_filtered

chr1.bed  chr1.bim  chr1.fam  chr1.log	chr1.nosex


We use PLINK to compute allele frequencies for variants present in our filtered genotype dataset, focusing on chromosome 1. The dataset is stored in the "ld_pruned_filtered/chr1" directory. The computed allele frequencies will be saved as an output file using the same directory structure and naming convention.

In [48]:
# Missing freq file
bash_command = """
bfile=ld_pruned_filtered/chr1
plink \
--bfile $bfile \
--freq \
--out $bfile
"""
run_bash_command(bash_command)


PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to ld_pruned_filtered/chr1.log.
Options in effect:
  --bfile ld_pruned_filtered/chr1
  --freq
  --out ld_pruned_filtered/chr1

515699 MB RAM detected; reserving 257849 MB for main workspace.
13300 variants loaded from .bim file.
3473 people (0 males, 0 females, 3473 ambiguous) loaded from .fam.
Ambiguous sex IDs written to ld_pruned_filtered/chr1.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 3473 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%5

In [46]:
!ls ld_pruned_filtered

chr1.bed  chr1.bim  chr1.fam  chr1.frq	chr1.log  chr1.nosex


We load the PLINK .frq file into a pandas DataFrame, extracted from our filtered genotype dataset focusing on chromosome 1. Adjustments are made to the DataFrame to conform to the required format, specifically selecting columns for SNP, allele, and minor allele frequency (MAF). Subsequently, we write this adjusted DataFrame to a new file 'ld_pruned_filtered/chr1.freq' in tab-separated format, omitting the index and header to ensure compatibility with downstream analysis tools.

In [49]:

# Load the PLINK .frq file
df = pd.read_csv('ld_pruned_filtered/chr1.frq', delim_whitespace=True)

# Adjust the DataFrame to match the required format
df_adjusted = df[['SNP', 'A1', 'MAF']]
df_adjusted.columns = ['SNP', 'Allele', 'Frequency']

# Write the adjusted DataFrame to a new file
df_adjusted.to_csv('ld_pruned_filtered/chr1.freq', sep='\t', index=False, header=False)


In [50]:
!ls ld_pruned_filtered

chr1.bed  chr1.bim  chr1.fam  chr1.freq  chr1.frq  chr1.log  chr1.nosex


We set the output directory to 'gctb_sparse_ldm', removing any existing directory and creating a new one if necessary. Then, we define the dataset using the filtered genotype data from chromosome 1 stored in 'ld_pruned_filtered/chr1'. Utilizing GCTB (Genome-wide Complex Trait Bayesian analysis), we compute the sparse version of the linkage disequilibrium matrix (--make-sparse-ldm) for variants with a linkage disequilibrium threshold of 0.2 (--ld 0.2). The resulting files will be stored in the specified output directory with the prefix 'chr1'. Finally, we display the first two lines of the 'chr1.ldm.sparse.info' file to inspect the sparse linkage disequilibrium matrix information.

In [55]:
# Example usage:
bash_command = """
output_dir=gctb_sparse_ldm
rm -rf $output_dir
mkdir -p $output_dir
bfile=ld_pruned_filtered/chr1
gctb_2.05beta_Linux/gctb_2.05beta_Linux/gctb \
--bfile $bfile \
--make-sparse-ldm \
--ld 0.2
--out "$output_dir/chr1"
head -n 2 "$output_dir/chr1.ldm.sparse.info"
"""
run_bash_command(bash_command)


******************************************************************
* GCTB 2.05beta                                                  *
* Genome-wide Complex Trait Bayesian analysis                    *
* Authors: Jian Zeng, Luke Lloyd-Jones, Zhili Zheng, Shouye Liu  *
* MIT License                                                    *
******************************************************************

Analysis started: Tue Mar 12 15:15:38 2024

Options:

--bfile ld_pruned_filtered/chr1
--make-sparse-ldm 
--wind 0.2
--out gctb_sparse_ldm/chr1

Reading PLINK FAM file from [ld_pruned_filtered/chr1.fam].
3473 individuals to be included from [ld_pruned_filtered/chr1.fam].
Reading phenotypes from [ld_pruned_filtered/chr1.fam].
Non-missing phenotypes of trait 1 of 3473 individuals are included from [ld_pruned_filtered/chr1.fam].
3473 matched individuals are kept.
Reading PLINK BIM file from [ld_pruned_filtered/chr1.bim].
13300 SNPs to be included from [ld_pruned_filtered/chr1.bim].
13300 SNPs o

We designate the output directory as 'gctb_full_ldm', ensuring any existing directory is removed and a new one is created if needed. Next, we define the dataset using the filtered genotype data from chromosome 1 stored in 'ld_pruned_filtered/chr1'. Employing GCTB, we compute the full version of the linkage disequilibrium matrix (--make-full-ldm) for variants with a linkage disequilibrium threshold of 0.1 (--ld 0.1). The resulting files will be stored in the specified output directory with the prefix 'chr1'.

In [60]:
# Trying full
bash_command = """
output_dir=gctb_full_ldm
rm -rf $output_dir
mkdir -p $output_dir
bfile=ld_pruned_filtered/chr1
gctb_2.05beta_Linux/gctb_2.05beta_Linux/gctb \
--bfile $bfile \
--make-full-ldm \
--ld 0.1
--out "$output_dir/chr1"

"""
run_bash_command(bash_command)

******************************************************************
* GCTB 2.05beta                                                  *
* Genome-wide Complex Trait Bayesian analysis                    *
* Authors: Jian Zeng, Luke Lloyd-Jones, Zhili Zheng, Shouye Liu  *
* MIT License                                                    *
******************************************************************

Analysis started: Tue Mar 12 15:23:22 2024

Options:

--bfile ld_pruned_filtered/chr1
--make-full-ldm 
--ld 0.1

Reading PLINK FAM file from [ld_pruned_filtered/chr1.fam].
3473 individuals to be included from [ld_pruned_filtered/chr1.fam].
Reading phenotypes from [ld_pruned_filtered/chr1.fam].
Non-missing phenotypes of trait 1 of 3473 individuals are included from [ld_pruned_filtered/chr1.fam].
3473 matched individuals are kept.
Reading PLINK BIM file from [ld_pruned_filtered/chr1.bim].
13300 SNPs to be included from [ld_pruned_filtered/chr1.bim].
13300 SNPs on 1 chromosomes are included.
B


We check the new generate file `gctb_full_ldm/chr1.ldm.full.info`.

The output provides detailed information about each variant included in the linkage disequilibrium matrix computed using GCTB. Here's a closer look at what each column signifies:

- **Chrom (Chromosome)**: Indicates the chromosome number where the variant is located.
- **ID (Variant Identifier)**: Uniquely identifies each variant in the dataset.
- **GenPos (Genetic Position)**: Represents the genetic position of the variant.
- **PhysPos (Physical Position)**: Denotes the physical position of the variant along the chromosome.
- **A1 (Allele 1)** and **A2 (Allele 2)**: Indicate the two alleles present at the variant locus.
- **A1Freq (Allele 1 Frequency)**: Specifies the frequency of Allele 1 in the sample population.
- **Index**: Index value assigned to the variant.
- **WindStart (Window Start)** and **WindEnd (Window End)**: Define the start and end positions of the genomic window associated with the variant.
- **WindSize (Window Size)**: Represents the size of the genomic window.
- **WindWidth (Window Width)**: Denotes the width of the genomic window.
- **N (Number of Samples)**: Indicates the total number of samples considered in the analysis.
- **SamplVar (Sample Variance)**: Represents the sample variance associated with the variant.
- **LDsum (Linkage Disequilibrium Sum)**: Provides information about the linkage disequilibrium sum for the variant.

This detailed output offers insights into the genetic characteristics of the variants, including their frequency distribution, physical and genetic positions, and their relationship with neighboring variants within the genomic window.

In [59]:
!head -n 10 gctb_full_ldm/chr1.ldm.full.info

 Chrom              ID     GenPos         PhysPos     A1     A2       A1Freq      Index  WindStart    WindEnd   WindSize       WindWidth          N     SamplVar        LDsum
     1      chr1:55365          0           55365      A      T     0.059603          0          0      13299      13300       282506249       3473         -nan         -nan
     1     chr1:275972          0          275972      A      T     0.489346          1          0      13299      13300       282506249       3473         -nan         -nan
     1     chr1:475912          0          475912      T      C     0.493953          2          0      13299      13300       282506249       3473         -nan         -nan
     1     chr1:759319          0          759319      C      T     0.466456          3          0      13299      13300       282506249       3473         -nan         -nan
     1     chr1:835524          0          835524      T      C     0.351137          4          0      13299      13300       282

In [32]:
%%bash
gctb_2.05beta_Linux/gctb_2.05beta_Linux/gctb --sbayes R \
--bfile ./data/bmi/gctb_genotype_data/P50_round2_LD_pruned_3473 \
--gwas-summary ./GCTA_COJO/physiological_bmi_bodylength_w_tail.ma

******************************************************************
* GCTB 2.05beta                                                  *
* Genome-wide Complex Trait Bayesian analysis                    *
* Authors: Jian Zeng, Luke Lloyd-Jones, Zhili Zheng, Shouye Liu  *
* MIT License                                                    *
******************************************************************

Analysis started: Tue Mar 12 03:21:39 2024

Options:

--sbayes R
--bfile ./data/bmi/gctb_genotype_data/P50_round2_LD_pruned_3473
--gwas-summary ./GCTA_COJO/physiological_bmi_bodylength_w_tail.ma

Reading PLINK FAM file from [./data/bmi/gctb_genotype_data/P50_round2_LD_pruned_3473.fam].
3473 individuals to be included from [./data/bmi/gctb_genotype_data/P50_round2_LD_pruned_3473.fam].
Reading PLINK BIM file from [./data/bmi/gctb_genotype_data/P50_round2_LD_pruned_3473.bim].
123673 SNPs to be included from [./data/bmi/gctb_genotype_data/P50_round2_LD_pruned_3473.bim].
Reading GWAS summary dat

bash: line 3:   432 Segmentation fault      (core dumped) gctb_2.05beta_Linux/gctb_2.05beta_Linux/gctb --sbayes R --bfile ./data/bmi/gctb_genotype_data/P50_round2_LD_pruned_3473 --gwas-summary ./GCTA_COJO/physiological_bmi_bodylength_w_tail.ma


CalledProcessError: Command 'b'gctb_2.05beta_Linux/gctb_2.05beta_Linux/gctb --sbayes R \\\n--bfile ./data/bmi/gctb_genotype_data/P50_round2_LD_pruned_3473 \\\n--gwas-summary ./GCTA_COJO/physiological_bmi_bodylength_w_tail.ma\n'' returned non-zero exit status 139.

In [50]:
%%bash
plink \
--bfile ./data/bmi/ld_pruned_genotype_data/LD_pruned_PLINK/P50_round2_LD_pruned_3473 \
--exclude gctb_snps_to_exclude.txt \
--make-bed \
--out bed_dir/out

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to bed_dir/out.log.
Options in effect:
  --bfile ./data/bmi/ld_pruned_genotype_data/LD_pruned_PLINK/P50_round2_LD_pruned_3473
  --exclude gctb_snps_to_exclude.txt
  --make-bed
  --out bed_dir/out

515699 MB RAM detected; reserving 257849 MB for main workspace.
128401 variants loaded from .bim file.
3473 people (0 males, 0 females, 3473 ambiguous) loaded from .fam.
Ambiguous sex IDs written to bed_dir/out.nosex .
--exclude: 71796 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 3473 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%4

In [51]:
# !mkdir ./bed_dir

In [52]:
# !plink --bfile ./data/bmi/gctb_genotype_data/P50_round2_LD_pruned_3473 --make-bed --out ./bed_dir/out

In [53]:
%%bash
gctb_2.05beta_Linux/gctb_2.05beta_Linux/gctb \
--bfile ./bed_dir/out \
--make-sparse-ldm \
--out custom_sparse_matrix_data/gctb

Process is terminated.


In [33]:
!mkdir custom_sparse_matrix_data

In [16]:
import pandas as pd

In [17]:
df = pd.read_csv('data/bmi/gwas_summary_files/physiological_bmi_bodylength_w_tail.csv')

In [18]:
df.head()

,chr,rs,ps,n_miss,allele1,allele0,af,beta,se,logl_H1,l_remle,l_mle,p_wald,p_lrt,p_score
0,chr10,chr10:149435,149435,0,T,C,0.876,-0.016290,0.042265,-4326.415,1.332775,1.333718,0.699948,0.699824,0.699854
1,chr10,chr10:767532,767532,0,C,T,0.611,0.006059,0.028462,-4326.467,1.332503,1.333593,0.831419,0.831345,0.831359
2,chr10,chr10:790368,790368,0,A,G,0.487,-0.004377,0.027795,-4326.477,1.332693,1.333571,0.874868,0.874812,0.874822
3,chr10,chr10:811265,811265,0,C,T,0.876,-0.014988,0.042217,-4326.427,1.332774,1.333704,0.722596,0.722480,0.722507
4,chr10,chr10:823144,823144,0,A,G,0.618,-0.003029,0.029571,-4326.485,1.332296,1.333446,0.918433,0.918427,0.918434


In [20]:
new_df = df[['ps', 'allele1', 'allele1', 'beta', 'se', 'p_score']]

In [21]:
new_df = new_df.rename(columns = {
    'ps': 'SNP',
    'allele0': 'A1',
    'allele1': 'A2',
    'af': 'freq',
    'beta': 'b',
    'p_score': 'p'
})

In [22]:
new_df['N'] = 3473

In [23]:
new_df

,SNP,A2,A2,b,se,p,N
0,149435,T,T,-0.016290,0.042265,0.699854,3473
1,767532,C,C,0.006059,0.028462,0.831359,3473
2,790368,A,A,-0.004377,0.027795,0.874822,3473
3,811265,C,C,-0.014988,0.042217,0.722507,3473
4,823144,A,A,-0.003029,0.029571,0.918434,3473
...,...,...,...,...,...,...,...
123577,99975834,C,C,0.020609,0.045388,0.649804,3473
123578,99987154,G,G,-0.059860,0.052312,0.252662,3473
123579,99989918,C,C,-0.088255,0.029432,0.002763,3473
123580,99993827,C,C,-0.081868,0.031653,0.009799,3473


In [24]:
new_df.to_csv('gctb_summary_stat.ma', index=False)